# CP-SAT: Decimal numbers

In [ ]:
from ortools.sat.python import cp_model

## Division with decimal numbers

CP-SAT is an integer programming solver. Thus, it only stores integer variables. 

But when you divide variables, you sometimes need more precision than an integer. 

Simply multiply the variable by a big number to get a rounded down approximation. 

In [ ]:
FLOAT_APPROX_PRECISION = 100

In [ ]:
model = cp_model.CpModel()

numerator = model.NewIntVar(0, 100, "numerator")
denominator = model.NewIntVar(1, 100, "denominator")
result = model.NewIntVar(0, 100, "result")

# We want to compute 10/30
model.Add(numerator == 10)
model.Add(denominator == 30)
# Add a divison equality and multiply numerator by 100
model.AddDivisionEquality(result, numerator * FLOAT_APPROX_PRECISION, denominator)

# Solve
solver = cp_model.CpSolver()
status = solver.Solve(model)

# Divide the result to get a rounded down solution
print(f"Solution is approximately: {solver.Value(result) / FLOAT_APPROX_PRECISION}")


Solution is approximately: 0.33


## Constraints with decimal numbers

Let's say you want the result variable to be more than half of the variable x. This will fail.

In [ ]:
model = cp_model.CpModel()

x = model.NewIntVar(0, 100, "x")
result = model.NewIntVar(0, 100, "result")

# Assume x == 30
model.Add(numerator == 30)
# This will fail
model.Add(result >= 0.5 * x)

TypeError: Not an integer: -0.5

But you can rewrite this inequality, to say that the variable result should be at least twice as big as the variable x.

In this casee, the constraint is perfectly valid.

In [ ]:
model.Add(2 * result >= x)

More generally, a quick and dirty way to handle constraints with decimal numbers is to multiply both sides by a big number and round everything. 

In [ ]:
# This will work well enough in most cases
model.Add(FLOAT_APPROX_PRECISION*result >= round(FLOAT_APPROX_PRECISION * 0.5) * x)

## Interpreting ints as bytes

In [90]:
# https://en.wikipedia.org/wiki/Minifloat

def int_to_minifloat(number: int):
    number_as_bytes = bin(number)[2:].rjust(8, "0")
    print("number_as_bytes", "".join([str(n) for n in number_as_bytes]))
    sign = number_as_bytes[0]
    if sign=="0":
        sign_value = 1
    else:
        sign_value = -1
    exponent = number_as_bytes[1:5]
    significand = number_as_bytes[-3:]
    # significand_value = int(significand.hex(), 8)
    significand_value = sum([int(v)*(1/2)**(i+1) for i, v in enumerate(significand)])
    exponent_value = int(exponent, 2)
    print("sign", sign_value, "e", exponent, exponent_value, "s", significand, significand_value)
    if exponent_value == 0:
        two_to_the_power = 0
        two_to_the_power_mult = 1
    else:
        two_to_the_power = 2**(exponent_value - 1)
        two_to_the_power_mult = 2**(exponent_value - 1)
    number_as_value = sign_value * (two_to_the_power + two_to_the_power_mult * significand_value)
    return number_as_value

for i in range(0, 255):
    print(int_to_minifloat(i))

number_as_bytes 00000000
sign 1 e 0000 0 s 000 0.0
0.0
number_as_bytes 00000001
sign 1 e 0000 0 s 001 0.125
0.125
number_as_bytes 00000010
sign 1 e 0000 0 s 010 0.25
0.25
number_as_bytes 00000011
sign 1 e 0000 0 s 011 0.375
0.375
number_as_bytes 00000100
sign 1 e 0000 0 s 100 0.5
0.5
number_as_bytes 00000101
sign 1 e 0000 0 s 101 0.625
0.625
number_as_bytes 00000110
sign 1 e 0000 0 s 110 0.75
0.75
number_as_bytes 00000111
sign 1 e 0000 0 s 111 0.875
0.875
number_as_bytes 00001000
sign 1 e 0001 1 s 000 0.0
1.0
number_as_bytes 00001001
sign 1 e 0001 1 s 001 0.125
1.125
number_as_bytes 00001010
sign 1 e 0001 1 s 010 0.25
1.25
number_as_bytes 00001011
sign 1 e 0001 1 s 011 0.375
1.375
number_as_bytes 00001100
sign 1 e 0001 1 s 100 0.5
1.5
number_as_bytes 00001101
sign 1 e 0001 1 s 101 0.625
1.625
number_as_bytes 00001110
sign 1 e 0001 1 s 110 0.75
1.75
number_as_bytes 00001111
sign 1 e 0001 1 s 111 0.875
1.875
number_as_bytes 00010000
sign 1 e 0010 2 s 000 0.0
2.0
number_as_bytes 00010001


'00000001'

1